In [7]:
from binance.client import Client
from Binance_api import api_key, secret_key
import pandas as pd
import time
from datetime import datetime
 

In [13]:
client = Client(api_key, secret_key)

In [23]:

coin = 'ETH'
start_str = 'JAN, 2022'

klines = client.get_historical_klines(symbol=f'{coin}USDT', interval=client.KLINE_INTERVAL_5MINUTE, start_str=start_str)

cols = ['OpenTime',
        f'{coin}-USD_Open',
        f'{coin}-USD_High',
        f'{coin}-USD_Low',
        f'{coin}-USD_Close',
        f'{coin}-USD_Volume',
        'CloseTime',
        f'{coin}-QuoteAssetVolume',
        f'{coin}-NumberOfTrades',
        f'{coin}-TBBAV',
        f'{coin}-TBQAV',
        f'{coin}-ignore']

coin_df = pd.DataFrame(klines,columns=cols)
coin_df['OpenTime'] = [datetime.fromtimestamp(ts / 1000) for ts in coin_df['OpenTime']]
coin_df['CloseTime'] = [datetime.fromtimestamp(ts / 1000) for ts in coin_df['CloseTime']]

coin_df.to_csv('ETH_5min_10JAN.csv')

In [12]:
coin_df

,OpenTime,BTC-USD_Open,BTC-USD_High,BTC-USD_Low,BTC-USD_Close,BTC-USD_Volume,CloseTime,BTC-QuoteAssetVolume,BTC-NumberOfTrades,BTC-TBBAV,BTC-TBQAV,BTC-ignore
0,2022-10-10 10:00:00,19439.96000000,19440.59000000,19407.83000000,19425.81000000,1509.92144000,2022-10-10 10:14:59.999,29328905.33949040,37687,749.10751000,14551207.05958230,0
1,2022-10-10 10:15:00,19425.29000000,19457.21000000,19401.00000000,19413.55000000,1603.93627000,2022-10-10 10:29:59.999,31167419.35334770,37548,810.72261000,15754353.23355760,0
2,2022-10-10 10:30:00,19414.17000000,19475.97000000,19403.83000000,19466.17000000,1474.47349000,2022-10-10 10:44:59.999,28655137.04246890,37502,790.84535000,15370693.00472850,0
3,2022-10-10 10:45:00,19466.17000000,19521.58000000,19462.00000000,19491.42000000,2479.69941000,2022-10-10 10:59:59.999,48335292.49871010,57508,1266.08712000,24680107.22239590,0
4,2022-10-10 11:00:00,19491.42000000,19521.84000000,19491.20000000,19496.59000000,1995.48412000,2022-10-10 11:14:59.999,38924277.30252410,45624,1031.28823000,20117510.87938840,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4732,2022-11-28 17:00:00,16213.36000000,16214.75000000,16190.24000000,16191.64000000,1781.46588000,2022-11-28 17:14:59.999,28863467.97695760,43463,788.18518000,12770678.14633760,0
4733,2022-11-28 17:15:00,16191.65000000,16211.33000000,16191.57000000,16203.78000000,1718.81036000,2022-11-28 17:29:59.999,27848995.37094740,40424,948.74152000,15372130.59661190,0
4734,2022-11-28 17:30:00,16203.78000000,16214.55000000,16194.02000000,16208.13000000,1686.08557000,2022-11-28 17:44:59.999,27321037.68936460,38244,912.90842000,14793062.77146970,0
4735,2022-11-28 17:45:00,16208.86000000,16235.20000000,16205.00000000,16222.06000000,1883.87498000,2022-11-28 17:59:59.999,30547946.99378490,43062,961.01087000,15583835.76392820,0


In [7]:
'''
innitially when price is within channel = none
when it crossed first bottom = hit
when it crosses above bottom after hit = released
'''

'\ninnitially when price is within channel = none\nwhen it crossed first bottom = hit\nwhen it crosses above bottom after hit = released\n'

In [26]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=0.99925, symbols=symbols)

for i in range(len(df)):  
    #means looking for something to buy
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if env.bottoms[symbol] == 'hit' and df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                env.bottoms[symbol] = 'released'
            
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] < 20: #buy signal
                if env.bottoms[symbol] == 'released':
                    env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                    env.reset_bottoms()
                    break
                else:
                    env.bottoms[symbol] = 'hit'
    
    #means looking to sell
    if env.balance_unit != 'USDT':
        if env.tops[env.balance_unit] == 'hit' and df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]:
            env.tops[env.balance_unit] = 'released'
        
        if df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] > 80: #sell signal
            if env.tops[env.balance_unit] == 'released':
                env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])
                env.reset_tops()
            else:
                env.tops[env.balance_unit] = 'hit'

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])
    
   

NameError: name 'TradingEnv' is not defined

In [23]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 4
num sells: 4
ending balance: 96.5748894792675 USDT


In [31]:
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    print('\n')

['BTC', '2022-11-16 10:56:00', 16792.25083691415]
['BTC', '2022-11-16 11:57:00', 16855.338406492894]
P/L = 0.37569454024626303


['BTC', '2022-11-16 14:22:00', 16928.967577869214]
['BTC', '2022-11-16 15:18:00', 16922.039155381153]
P/L = -0.0409264324962022


['BTC', '2022-11-16 16:11:00', 16889.292258865134]
['BTC', '2022-11-16 17:02:00', 16896.161880173106]
P/L = 0.04067441786594905


['BTC', '2022-11-16 18:00:00', 16815.619922646805]
['BTC', '2022-11-16 20:04:00', 16773.04289935849]
P/L = -0.2531992485806267


['BTC', '2022-11-16 20:36:00', 16691.752089485755]
['BTC', '2022-11-16 20:43:00', 16754.65165393053]
P/L = 0.37683020995977756


['BTC', '2022-11-16 21:49:00', 16709.191793541515]
['BTC', '2022-11-16 23:33:00', 16674.15172413135]
P/L = -0.20970535165986126


['BTC', '2022-11-16 23:56:00', 16474.124285860224]
['BTC', '2022-11-17 00:34:00', 16578.37796267964]
P/L = 0.6328328899940061


['BTC', '2022-11-17 01:46:00', 16490.00050053416]
['BTC', '2022-11-17 02:41:00', 16505.97528804